### Colab instructions

In [ ]:
!git clone https://github.com/GioC1810/aml23-ego.git
!pip install omegaconf coloredlogs wandb

In [ ]:
# Mount google drive 
from google.colab import drive
drive.mount('/content/drive')

## Epic kitchen

#### Feature extraction script

The script extract the features for 5, 10 and 25 num frames per clip

In [ ]:
%%bash

splits=("test")
num_frames=(5)
dense_sampling=("True")

for split_type in ${splits[@]}
do
  for frames in ${num_frames[@]}
  do
    for sampling in ${dense_sampling[@]}
    do
        python3 save_feat.py \
            config=configs/I3D_save_feat.yaml \
            dataset.shift=D1-D1 \
            save.num_frames_per_clip.RGB=$frames \
            save.dense_sampling.RGB=$sampling \
            split=$split_type \
            dataset.RGB.data_path=./ek_data/frames 
    done
  done
done

#### Model training

Indicate:
- shift 
- num frames per clip  
- model 
- model name

In [ ]:
%%bash

python3 train_classifier_ek.py name=model_name \
  config=configs/default.yaml \
  dataset.shift=D1-D1 \
  train.num_frames_per_clip.RGB=25 \
  train.dense_sampling.RGB=False \
  models.RGB.model=MLP_classifier

## Action net

#### Action net frame extraction

In [ ]:
%%bash

ffmpeg -i action-net/video/S04_video.mp4 -vf "fps=30,scale=456:256" -q:v 2 action-net/frames/S04_1/frame_%010d.jpg

#### Feature rgb extraction
Indicate:
- num frames per clip
- split (train or test)
- dense sampling (True or False)

In [ ]:
%%bash

python3 save_feat_action_net.py \
  config=configs/feature_rgb_extraction.yaml \
  save.dense_sampling.RGB=True \
  save.num_frames_per_clip.RGB=5 \
  split=test

#### Decompress emg features train

In order to push the features we compressed them, this script decompress them

In [18]:
import lzma

with lzma.open('saved_features_an_multimodal/features_emg_allData_train.pkl.xz', 'rb') as f_in:
    with open('saved_features_an_multimodal/features_emg_allData_train.pkl', 'wb') as f_out:
        f_out.write(f_in.read())

#### Split emg spectogram features in order to push it

This is the script to split the features in 3 different files in order to push to the repo

In [9]:
import pickle

def split_dict(filename, output_prefix):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    
    length = len(data['features'])
    third_length = length // 3

    split = [data['features'][0:third_length], data['features'][third_length:2*third_length], data['features'][2*third_length:length]]

    for i in range(1,4):
        with open(output_prefix + f'_{i}.pkl', 'wb') as f:
            pickle.dump(split[i-1], f)

split_dict('saved_features_an_multimodal/features_emg_spectrogram_train.pkl', 'saved_features_an_multimodal/features_emg_spectrogram_train_split')

#### Decompress and join emg spectogram features

This is the script to execute in order to obtain the joined features

In [2]:
import pickle

def join_spectogram_feature(filename, output_name):
    joined_features = {'features': []}
    
    for i in range(3):
        with open(filename + f'_{i+1}.pkl', 'rb') as f:
            joined_features['features'].extend(pickle.load(f))
            
    
    with open(output_name, 'wb') as f:
        pickle.dump(joined_features, f)
        
join_spectogram_feature('saved_features_an_multimodal/features_emg_spectrogram_train_split', 
                        'saved_features_an_multimodal/features_emg_spectrogram_train.pkl')

#### Benchmarking EMG

To test the different performance of the EMG classifier according to the different combinations of the EMG preprocessing parameters, and to avoid computing locally the features and push them on github here the code to compute them on colab. 

- Preprocessing
- Reformatting the features as the loader expects them


In [ ]:
%%bash
cd aml23-ego/EMG
python EMG_preprocessing.py

In [ ]:
import pickle
import os
import pandas as pd


def create_emg_features(full_data, split, spectrogram):
    #os.chdir('aml23-ego/EMG/')
    full_data = pd.read_pickle(full_data)
    print(len(full_data))
    
    #full_data = full_data[full_data['file'] == 'S04_1.pkl']
    full_data = full_data.rename(columns={'emg_data': 'features_EMG'})

    emg_features = full_data[
        ['uid', 'features_EMG']]
    emg_features = emg_features.to_dict(orient='index')
    emg_features = {'features': list(emg_features.values())}
    print(emg_features['features'][0]['features_EMG'].shape)
    features_name = f'../saved_features_an_multimodal/features_emg_spectogram_allData_{split}.pkl' if spectrogram \
        else f'../saved_features_an_multimodal/features_emg_10fs_10s_30seg_allData_{split}.pkl'

    with open(features_name, 'wb') as f:
        pickle.dump(emg_features, f)

create_emg_features(full_data="./new_emg_data_10fs_10s_30seg_train.pkl", split='train', spectrogram=False)
create_emg_features(full_data="./new_emg_data_10fs_10s_30seg_test.pkl", split='test', spectrogram=False)

#### Model training

###### RGB model
Indicate:
- Dense sampling: to choose which features the model use for training
- num frames per clip: to choose which features the model use for training
- model: which model to use

In [ ]:
%%bash

python3 train_classifier.py name=train_model_rgb \
  config=configs/training_rgb.yaml \
  train.dense_sampling.RGB=True \
  train.num_frames_per_clip.RGB=25 \
  models.RGB.model=TRN_classifier

###### EMG model

Indicate:
- spectogram_feat: To choose which kind of features the model is train on
- model: according to the kind of modality 
    - spectogram_feat = True -> LeNet5
    - spectogram_feat = False -> EMG_LSTM

In [ ]:
%%bash

python3 train_classifier_multimodal.py name=train_lstm_emg\
  config=configs/training_emg.yaml \
  train.spectogram_feat=False \
  models.EMG.model=EMG_LSTM

#### Multimodal models

##### Late fusion model

###### Testing

Indicate:
- Dense sampling: to choose which features the model use for training
- num frames per clip: to choose which features the model use for training
- model: which model to use (both RGB and EMG)
- spectogram_feat: To choose which kind of features the model is train on


In [ ]:
%%bash

python3 train_classifier_multimodal.py name=multimodal_late \
  config=configs/late_fusion_test.yaml \
  train.dense_sampling.RGB=True \
  train.num_frames_per_clip.RGB=25 \
  train.spectogram_feat=False \
  models.RGB.model=TRN_classifier \
  models.EMG.model=EMG_LSTM

##### Midl level fusion model

###### Mid level feature extraction

Indicate: 
- Dense sampling: to choose which features the model use for training
- num frames per clip: to choose which features the model use for training
- split (train or test)
- model: which model to use (both RGB and EMG)
- spectogram_feat: To choose which kind of features the model is train on

In [ ]:
%%bash

python3 save_feat_midlevel_multimodal.py name=midlevel_feat \
  config=configs/mid_level_extraction.yaml \
  split=train \
  save.dense_sampling.RGB=True \
  save.num_frames_per_clip.RGB=25 \
  save.spectogram_feat=False \
  models.RGB.model=Lstm_classifier \
  models.EMG.model=EMG_LSTM

###### Training

Indicate: 
- Dense sampling: to choose which features the model use for training
- num frames per clip: to choose which features the model use for training
- model: which model to use (both RGB and EMG)
- spectogram_feat: To choose which kind of features the model is train on

In [ ]:
%%bash

python3 train_midlevel_classifier.py name=midlevel_classifier \
  config=configs/multimodal_midLevel_training.yaml \
  train.dense_sampling.RGB=True \
  train.num_frames_per_clip.RGB=25 \
  train.spectogram_feat=False \
  models.RGB.model=Lstm_classifier \
  models.EMG.model=EMG_LSTM